# MNIST Digits - Classification Using SVM

In this notebook, we'll explore the popular MNIST dataset and build an SVM model to classify handwritten digits

## Step1: Data Understanding

In [30]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

pd.options.display.max_rows=None
pd.options.display.max_columns=None

for directory,subdir,files in os.walk('../input'):   
    for file in files:
        print(os.path.join(directory,file))
    

In [92]:
# read the csv to pandas dataframe
digits_train_df = pd.read_csv('../input/digit-recognizer/train.csv')
digits_test_df = pd.read_csv('../input/digit-recognizer/test.csv')
#check the number of rows and columns
print("Train DF Shape",digits_train_df.shape)
print("Test DF Shape", digits_test_df.shape)

In [ ]:
digits_test_df.head()

In [5]:
digits_train_df.head()

In [6]:
# check the info about each column dtype, non-null count
digits_train_df.info(verbose=True, show_counts=True)

In [7]:
#check null counts
digits_train_df.loc[:,digits_train_df.isna().sum()>0].columns

In [8]:
# Let's plot the digits from 0 to 9 which are in train df
fig, ax = plt.subplots(2,5,figsize=(15,8))
row=0
col=0
for i in range(10):
    if i==5:
        row=1
        col=0
    ax[row,col].imshow(digits_train_df[digits_train_df['label']==i].iloc[0,1:].values.reshape(28,28),cmap='gray')
    col+=1
    fig.show()

In [9]:
digits_train_df.describe()

## Step 2: Dapa Preparation and Model Building

In [10]:
# As dataset has 40k+ rows It will take some hours if we try to fit whole data. So let's take 10% of data

digits_train_10_df, digits_test_90_df = train_test_split(digits_train_df, train_size=0.1,random_state=1000)
y_train = digits_train_10_df['label']
X_train = digits_train_10_df.drop(['label'],axis=1)

In [11]:
standard_scaler = StandardScaler()
X_train = standard_scaler.fit_transform(X_train)

In [88]:
## Transform Test data also with scaler
X_test = standard_scaler.transform(digits_test_df)

#### Grid Search Cross Validation

In [12]:
# creating 5 folds and with gamma and C params
folds = KFold(n_splits=5, shuffle=True, random_state=1000)
params = {"gamma":[1e-2,1e-3,1e-4],
          "C":[1,10,100,1000]}

svc_grid_search_model = SVC(kernel='rbf')
classifier_model = GridSearchCV(estimator=svc_grid_search_model,
                               param_grid=params,
                               scoring='accuracy',
                               cv=folds,
                               return_train_score=True
                               )
classifier_model.fit(X_train,y_train)

In [13]:
## check best score and params
print(classifier_model.best_params_)
print(classifier_model.best_score_)

In [15]:
classifier_model_results = pd.DataFrame(classifier_model.cv_results_)

In [16]:
classifier_model_results.head()

In [20]:
gamma_01 = classifier_model_results[classifier_model_results['param_gamma']==0.01]
gamma_001 = classifier_model_results[classifier_model_results['param_gamma']==0.001]
gamma_0001 = classifier_model_results[classifier_model_results['param_gamma']==0.0001]

In [26]:
gamma_01.head()

In [84]:
gamma = [1e-2,1e-3,1e-4]
fig = make_subplots(rows=1, cols=3, shared_yaxes=True, subplot_titles=('Gamma_0.01','Gamma_0.001','Gamma_0.0001'),y_title='Accuracy')
col=1
for gamma_val in gamma:
    gamma_results = classifier_model_results[classifier_model_results['param_gamma']==gamma_val]
    tr1 = go.Scatter(x=gamma_results['param_C'],y=gamma_results['mean_test_score'],name='Train Score')
    tr2 = go.Scatter(x=gamma_results['param_C'],y=gamma_results['mean_train_score'],name='Test Score')
    fig.add_trace(tr1,row=1,col=col)
    fig.add_trace(tr2,row=1,col=col)
    col+=1
fig.update_xaxes(title={'text':'Cost-C'})
fig.update_yaxes(tickvals=[0.6, 0.7, 0.8,0.9,1,1.1])
fig.show()
    
    


#### Observations

We can see from above graphs below are best params
{'C': 100, 'gamma': 0.001}


### Final Model

In [86]:
C=100
gamma=0.001

svm_model = SVC(kernel='rbf',C=C, gamma=gamma)
svm_model.fit(X_train,y_train)

In [89]:
y_test_pred = svm_model.predict(X_test)

In [90]:
y_test_pred[0:5]

In [106]:
ImageId = np.arange(1,28001).reshape(-1)

In [109]:
ImageId[0:5]

In [107]:
type(ImageId)

In [111]:
submission = pd.DataFrame(zip(ImageId,y_test_pred),columns=['ImageId','Label'])

submission.to_csv("submission.csv",index=False)